In [0]:
import delta

In [0]:
df_full = spark.read.format('parquet').load('/Volumes/raw/full_load/customers/')
display(df_full)

In [0]:
(df_full.coalesce(1)
    .write.format("delta")
    .mode("overwrite")
    .saveAsTable(
    "bronze.upsell.customers"
)
    )

In [0]:
%%sql

select * from bronze.upsell.customers

In [0]:
spark.read \
    .format("parquet") \
    .load("/Volumes/raw/data/cdc/customers/") \
    .createOrReplaceTempView("customers_cdc")

In [0]:
query = """
    select *
    from customers_cdc
    qualify row_number() over (partition by idCliente order by DtAtualizacao desc) = 1
"""

df_cdc_unique = spark.sql(query)
df_cdc_unique.display()

In [0]:
bronze = delta.DeltaTable.forName(spark, 'bronze.upsell.customers')


In [0]:
bronze.alias('b') \
        .merge(df_cdc_unique.alias('d'), 'b.IdCliente = d.IdCliente') \
        .whenMatchedDelete(condition="d.op = 'D'") \
        .whenMatchedUpdateAll(condition="d.op = 'U'") \
        .whenNotMatchedInsertAll(condition="d.op = 'I' OR d.op = 'U'") \
        .execute()

In [0]:
%sql
select * from bronze.upsell.customers